In [1]:
#Steven Kim, Arun Anand
#Code adapted from the blockchain tutorial found on http://ecomunsing.com/build-your-own-blockchain?fbclid=IwAR0wwxa9kU_GXasSnotvGmbKuEqHqYdxHal6lHPUz0IoK1NktE0I99CTl7o
import hashlib, json, sys
import random


#This is a helper function for the hashing algorithm
def hashMe(msg=""):
    if type(msg)!=str:
        msg = json.dumps(msg,sort_keys=True)  # Sorting the keys to guarantee repeatability
        
    if sys.version_info.major == 2:
        return unicode(hashlib.sha256(msg).hexdigest(),'utf-8')
    else:
        return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

    
random.seed(0)
#This function is used to create valid transactions of random sign between 1 and maxValue (3 in this case)
def makeTransaction(maxValue=3):
    sign      = int(random.getrandbits(1))*2 - 1   # This will randomly choose -1 or 1 for the sign
    amount    = random.randint(1,maxValue)
    alicePays = sign * amount
    bobPays   = -1 * alicePays
    return {u'Alice':alicePays,u'Bob':bobPays}


#Testing the system by making 30 transactions
txnBuffer = [makeTransaction() for i in range(30)]



#takes in the transfer amount and the state 
#returns updated state
def updateState(txn, state):
    state = state.copy() 
    for key in txn:
        if key in state.keys():
            state[key] += txn[key]
        else:
            state[key] = txn[key]
    return state


#Set of rules for a valid transaction:
#Withdrawals must equal deposits
#Every withdrawal must be less than or equal to the balance
def isValidTxn(txn,state):
    if sum(txn.values()) is not 0:
        return False
    
    for key in txn.keys():
        if key in state.keys(): 
            acctBalance = state[key]
        else:
            acctBalance = 0
        if (acctBalance + txn[key]) < 0:
            return False
    
    return True


def makeBlock(txns,chain):
    parentBlock = chain[-1]
    parentHash  = parentBlock[u'hash']
    blockNumber = parentBlock[u'contents'][u'blockNumber'] + 1
    txnCount    = len(txns)
    blockContents = {u'blockNumber':blockNumber,u'parentHash':parentHash,
                     u'txnCount':len(txns),'txns':txns}
    blockHash = hashMe( blockContents )
    block = {u'hash':blockHash,u'contents':blockContents}
    
    return block




############## test cases ###########
state = {u'Alice':5,u'Bob':5}

print(isValidTxn({u'Alice': -3, u'Bob': 3},state))  # Basic transaction
print(isValidTxn({u'Alice': -4, u'Bob': 3},state))  #Invalid case, deposits != withdrawals
print(isValidTxn({u'Alice': -6, u'Bob': 6},state))  # Invalid test case, overdraft
print(isValidTxn({u'Alice': -4, u'Bob': 2,'Mallory':2},state)) #Valid test case
print(isValidTxn({u'Alice': -4, u'Bob': 3,'Mallory':2},state)) #Invalid test case, deposits != withdrawals 

#new testcases 
state = {u'Alice':50, u'Bob':50}
genesisBlockTxns = [state]
genesisBlockContents = {u'blockNumber':0,u'parentHash':None,u'txnCount':1,u'txns':genesisBlockTxns}
genesisHash = hashMe( genesisBlockContents )
genesisBlock = {u'hash':genesisHash,u'contents':genesisBlockContents}
genesisBlockStr = json.dumps(genesisBlock, sort_keys=True)

chain = [genesisBlock]



blockSizeLimit = 5 #Defines number of transactions per block

while len(txnBuffer) > 0:
    bufferStartSize = len(txnBuffer)
    
    ## This block waits while the appropriate number of transactions are gathered to fill up a block.
    txnList = []
    while (len(txnBuffer) > 0) & (len(txnList) < blockSizeLimit):
        newTxn = txnBuffer.pop()
        validTxn = isValidTxn(newTxn,state) 
        
        if validTxn:        
            txnList.append(newTxn)
            state = updateState(newTxn,state)
        else:
            print("ignored transaction")
            sys.stdout.flush()
            continu
    
    myBlock = makeBlock(txnList,chain)
    chain.append(myBlock)
    






True
False
False
True
False


In [7]:
chain[0]

{'hash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
 'contents': {'blockNumber': 0,
  'parentHash': None,
  'txnCount': 1,
  'txns': [{'Alice': 50, 'Bob': 50}]}}

In [8]:
chain[1]

{'hash': '7a91fc8206c5351293fd11200b33b7192e87fad6545504068a51aba868bc6f72',
 'contents': {'blockNumber': 1,
  'parentHash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
  'txnCount': 5,
  'txns': [{'Alice': 3, 'Bob': -3},
   {'Alice': -1, 'Bob': 1},
   {'Alice': 3, 'Bob': -3},
   {'Alice': -2, 'Bob': 2},
   {'Alice': 3, 'Bob': -3}]}}

In [9]:
state

{'Alice': 72, 'Bob': 28}

In [11]:
def checkBlockHash(block):
    Compare the hash against the contents of the block and raise an exception in the case of a discrepancy.
    expectedHash = hashMe( block['contents'] )
    if block['hash']!=expectedHash:
        raise Exception('Hash does not match contents of block %s'%
                        block['contents']['blockNumber'])
    return

In [ ]:
def checkBlockValidity(block,parent,state):    
    #Validate the blocks by making sure each transaction is valid, block hash is valid, block number = parent block +1, and contains valid reference to parent block hash.
    parentNumber = parent['contents']['blockNumber']
    parentHash   = parent['hash']
    blockNumber  = block['contents']['blockNumber']
    
    #This code block checks to make sure all transactions within the block are valid.
    for txn in block['contents']['txns']:
        if isValidTxn(txn,state):
            state = updateState(txn,state)
        else:
            raise Exception('Invalid transaction in block %s: %s'%(blockNumber,txn))

    checkBlockHash(block) #Use helper function above to validate block hash against the content (cryptographic security)

    if blockNumber!=(parentNumber+1):
        raise Exception('Hash does not match contents of block %s'%blockNumber)

    if block['contents']['parentHash'] != parentHash:
        raise Exception('Parent hash not accurate at block %s'%blockNumber)
    
    return state

In [12]:
def checkChain(chain):
    # This function checks the chain of blocks to check for transaction vaidity, and agreement between each block and its hash.
  
    
    if type(chain)==str:
        try:
            chain = json.loads(chain)
            assert( type(chain)==list)
        except:  # This is a catch-all, admittedly crude
            return False
    elif type(chain)!=list:
        return False
    
    state = {} 
    
    #Checking the genesis block (first block in the chain)
    for txn in chain[0]['contents']['txns']:
        state = updateState(txn,state)
    checkBlockHash(chain[0])
    parent = chain[0]
    
  #Routine to check all other blocks using the helper method above.
    for block in chain[1:]:
        state = checkBlockValidity(block,parent,state)
        parent = block
        
    return state

In [13]:
checkChain(chain)

{'Alice': 72, 'Bob': 28}

In [14]:
chainAsText = json.dumps(chain,sort_keys=True)
checkChain(chainAsText)

{'Alice': 72, 'Bob': 28}

In [15]:
import copy
nodeBchain = copy.copy(chain)
nodeBtxns  = [makeTransaction() for i in range(5)]
newBlock   = makeBlock(nodeBtxns,nodeBchain)

In [16]:
print("Blockchain on Node A is currently %s blocks long"%len(chain))

try:
    print("New Block Received; checking validity...")
    state = checkBlockValidity(newBlock,chain[-1],state) # Update the state- this will throw an error if the block is invalid!
    chain.append(newBlock)
except:
    print("Invalid block; ignoring and waiting for the next block...")

print("Blockchain on Node A is now %s blocks long"%len(chain))

Blockchain on Node A is currently 7 blocks long
New Block Received; checking validity...
Blockchain on Node A is now 8 blocks long
